In [ ]:
# default_exp models.mqcnn.mqcnn

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# MQ-CNN(RNN): A Multi-Horizon Quantile Recurrent Forecaster

> API details.
- https://arxiv.org/abs/1711.11053

In [ ]:
class MQCNN(object):
    """ Multi Quantile Convolutional Neural Network
    """
    def __init__(self,
             input_size,
             output_size,
             learning_rate,
             lr_scheduler_step_size,
             lr_decay,
             n_iterations,
             early_stopping,
             loss,
             val_loss,
             random_seed,
             device=None):
        pass